In [ ]:
BASE_PATH = "../input/mens-march-mania-2022/MDataFiles_Stage2/"
#stage1 BASE_PATH = "../input/mens-march-mania-2022/MDataFiles_Stage1/"
RESULTS_NAME = "MNCAATourneyDetailedResults.csv"
SEED_NAME = "MNCAATourneySeeds.csv"
RANK_NAME = "MMasseyOrdinals_thruDay128.csv"
#stage1 RANK_NAME = "MMasseyOrdinals.csv"

#stage1 TEST_NAME = "MSampleSubmissionStage1.csv"
TEST_NAME = "MSampleSubmissionStage2.csv"

#https://github.com/rishiraj/autolgbm
#LightGBM + Optuna: no brainer
#auto train lightgbm directly from CSV files
#auto tune lightgbm using optuna
#auto serve best lightgbm model using fastapi
model_type = 'Autolgbm'  
message='baseline'

train_filename = "./train.csv"
output = "output"
test_filename = "./test.csv"
task = None
targets = ['target']
features = None
categorical_features = None
use_gpu = False
num_folds = 10
seed = 42
num_trials = 10000
time_limit = 7200
fast = False

TEST = False

In [ ]:
if TEST:
    time_limit = 1000

In [ ]:
!pip install autolgbm

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pylab as plt
import seaborn as sns
from autolgbm import AutoLGBM

In [ ]:
!ls ../input/mens-march-mania-2022/MDataFiles_Stage1

In [ ]:
BASE_PATH + RESULTS_NAME

In [ ]:
df_TDresults = pd.read_csv(BASE_PATH + RESULTS_NAME)
# df_RSDresults = pd.read_csv("../input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv")
df_TDresults.head()

In [ ]:
# df_TDresults = pd.merge(df_RSDresults,df_TDresults,on = df_TDresults.columns.tolist(),how = "outer")
len(df_TDresults)

In [ ]:
df_TDresults.dtypes

In [ ]:
box_col = ["FGM","FGA","FGM3","FTM","FTA","OR","DR","Ast","TO","Stl","Blk","PF"]  #含各种统计数，没有score与target
df_boxW = df_TDresults[["Season","WTeamID"]+["W" + col for col in box_col]]
df_boxL = df_TDresults[["Season","LTeamID"]+["L" + col for col in box_col]]
df_boxW = df_boxW.rename(columns={"WTeamID":"TeamID"})
df_boxW = df_boxW.rename(columns={("W"+ col):col for col in box_col})
df_boxL = df_boxL.rename(columns={"LTeamID":"TeamID"})
df_boxL = df_boxL.rename(columns={("L"+ col):col for col in box_col})
df_box = pd.merge(df_boxW,df_boxL,on = ["Season","TeamID"]+box_col,how="outer")
df_box = df_box.groupby(["Season","TeamID"])[box_col].agg(np.mean).reset_index()
df_box.head()

In [ ]:
df_TDresults2 = df_TDresults
df_TDresults = df_TDresults.rename(columns={"WTeamID":"Team1ID","LTeamID":"Team2ID","WScore":"T1Score","LScore":"T2Score"})
df_TDresults = df_TDresults.rename(columns={f"W{col}":f"T1{col}" for col in box_col})
df_TDresults = df_TDresults.rename(columns={f"L{col}":f"T2{col}" for col in box_col})
df_TDresults2 = df_TDresults2.rename(columns={"WTeamID":"Team2ID","LTeamID":"Team1ID","WScore":"T2Score","LScore":"T1Score"})

In [ ]:
features = ["Season","Team1ID","Team2ID","T1Score","T2Score",'target']  #不含各种统计数，只有score与target
df_TDresults['target'] = 1.0
df_TDresults2['target'] = 0.0
train = pd.merge(df_TDresults,df_TDresults2,on = features,how="outer")
train = train[features]
train.head()

In [ ]:
box_T1 = df_box.copy()
box_T2 = df_box.copy()
box_T1.columns = ['Season','Team1ID'] + ["T1"+col+"_mean" for col in box_col]
box_T2.columns = ['Season','Team2ID'] + ["T2"+col+"_mean" for col in box_col]
train = pd.merge(train,box_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,box_T2,on = ["Season","Team2ID"],how = "left")

In [ ]:
#str(len(train))+","+str(len(df_box))
#train.head()
#len(train.where(train["Season"]==2013))
train.loc[train["target"]==1].head()

In [ ]:
BASE_PATH + SEED_NAME

In [ ]:
df_seeds = pd.read_csv(BASE_PATH + SEED_NAME)
df_seeds.head()

In [ ]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T2 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','Team1ID','T1_seed']
seeds_T2.columns = ['Season','Team2ID','T2_seed']
train = pd.merge(train,seeds_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,seeds_T2,on = ["Season","Team2ID"],how = "left")
train["seeddiff"] = train["T1_seed"] - train["T2_seed"]

In [ ]:
#len(train)
train.head()

In [ ]:
BASE_PATH + RANK_NAME

In [ ]:
df_MMOrdinals = pd.read_csv(BASE_PATH + RANK_NAME)
df_rank = df_MMOrdinals.groupby(["Season","TeamID"])["OrdinalRank"].agg(np.mean).reset_index()
df_rank.head()

In [ ]:
ranks_T1 = df_rank.copy()
ranks_T2 = df_rank.copy()
ranks_T1.columns = ['Season','Team1ID','T1_rank_mean']
ranks_T2.columns = ['Season','Team2ID','T2_rank_mean']
train = pd.merge(train,ranks_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,ranks_T2,on = ["Season","Team2ID"],how = "left")
train["rankdiff"] = train["T1_rank_mean"] - train["T2_rank_mean"]

In [ ]:
train = train.drop(columns = ["T1Score","T2Score"])

In [ ]:
train.head()

In [ ]:
BASE_PATH + TEST_NAME

In [ ]:
test = pd.read_csv(BASE_PATH + TEST_NAME)
test.head()

In [ ]:
test["Season"] = test['ID'].apply(lambda x: int(x[0:4]))
test["Team1ID"] = test['ID'].apply(lambda x: int(x[5:9]))
test["Team2ID"] = test['ID'].apply(lambda x: int(x[10:14]))
test.head()

In [ ]:
box_T1['Season'] = box_T1['Season'] + 1
box_T2['Season'] = box_T2['Season'] + 1

In [ ]:
box_T2[box_T2['Season']==2022].shape

In [ ]:
test = pd.merge(test,box_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,box_T2,on = ["Season","Team2ID"],how = "left")
test.head()

In [ ]:
test = pd.merge(test,seeds_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,seeds_T2,on = ["Season","Team2ID"],how = "left")
test["seeddiff"] = test["T1_seed"] - test["T2_seed"]

In [ ]:
test = pd.merge(test,ranks_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,ranks_T2,on = ["Season","Team2ID"],how = "left")
test["rankdiff"] = test["T1_rank_mean"] - test["T2_rank_mean"]

In [ ]:
test = test.drop(columns = ['ID','Pred'])
test.head()

In [ ]:
train.to_csv("train.csv",index = None)
test.to_csv("test.csv",index = None)

In [ ]:
features=None

In [ ]:
"train_filename:"+train_filename+",output:"+output+",test_filename:"+test_filename+","+str(task)+","+str(targets)+","+str(features)+","+str(categorical_features)+","+str(use_gpu)+","+str(num_folds)+","+str(seed)+","+str(num_trials)+","+str(time_limit)+","+str(fast)

In [ ]:
algbm = AutoLGBM(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    targets=targets,
    features=features,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)

# 使用2022赛季数据作为test数据集，并提交结果

In [ ]:
#才2000多个samples，单cpu需要2个小时
#algbm.train()

In [ ]:
#autolgb_pred = pd.read_csv("./output/test_predictions.csv")
#autolgb_pred.head()

In [ ]:
submission = pd.read_csv(BASE_PATH + TEST_NAME)
#autolgb_pred = pd.read_csv("./output/test_predictions.csv")
#submission['Pred'] = autolgb_pred['1.0']
# submission.rename(columns = {'1.0':'Pred'}, inplace = True)
submission.to_csv("submission.csv", index=False)
submission.head()

# 从train数据集按照比例分解为新的train与test数据集，并分析训练结果

In [ ]:
test_year = 2021
# 新train数据集
new_train = train[train['Season']!=test_year].copy().reset_index()
# 新test数据集
new_test = train[train['Season']==test_year].drop(columns = ["target"]).copy().reset_index()

print('全部数据集（正反2倍的seed）：'+str(len(train))+'，新训练集：'+str(len(new_train))+'，新测试集：'+str(len(new_test)))


In [ ]:
new_test.shape

In [ ]:
ttt = train[train['Season']==test_year][["Season","Team1ID","Team2ID","target"]]
ttt[(ttt['target']==0)&(ttt['Season']==2021)].head()

In [ ]:
test.head()

In [ ]:
new_test.head()

In [ ]:
new_train.to_csv("new_train.csv",index = None)
new_test.to_csv("new_test.csv",index = None)
new_train_filename = "./new_train.csv"
new_output = "new_output"
new_test_filename = "./new_test.csv"

In [ ]:
new_algbm = AutoLGBM(
    train_filename=new_train_filename,
    output=new_output,
    test_filename=new_test_filename,
    task=task,
    targets=targets,
    features=features,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)

In [ ]:
#new_algbm.train()

In [ ]:
new_autolgb_pred = pd.read_csv("./new_output/test_predictions.csv")
new_autolgb_pred.head()

In [ ]:
new_autolgb_pred.shape

In [ ]:
new_test['Pred'] = new_autolgb_pred['1.0']
new_test = new_test[["Season","Team1ID","Team2ID","Pred"]].reset_index()
new_test[(new_test['Team1ID']==1455)&(new_test['Team2ID']==1179)].head()

In [ ]:
new_test.shape

In [ ]:
#取出test赛季数据，验证训练效果
new_valid = train[train['Season']==test_year][["Season","Team1ID","Team2ID","target"]].reset_index()
new_valid[(new_valid['target']==0)].head()

In [ ]:
new_valid.shape

In [ ]:
new_check = pd.merge(new_test,new_valid,on = ["Season","Team1ID","Team2ID"],how="inner").reset_index()
new_check[(new_check['target']==0)].head()

In [ ]:
new_check.shape

In [ ]:
#计算均方根误差RMSE root mean square error
from sklearn.metrics import mean_squared_error
y_pred = new_check['Pred'].to_numpy()
y_test = new_check['target'].to_numpy()
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# 使用lightGBM训练，并分析预测结果

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [ ]:
test_year = 2017
pred_year = 2021
# 新train数据集
my_train = train[(train['Season']!=test_year)&(train['Season']!=pred_year)].copy().reset_index()
# 新test数据集
my_test = train[train['Season']==test_year].copy().reset_index()
# predict数据集
my_pred = train[train['Season']==pred_year].copy().reset_index()
my_test.head()

In [ ]:
#pandas转numpy
my_train_x = my_train.drop(columns = ['target','index']).to_numpy()
my_train_y = my_train['target'].to_numpy()
my_test_x = my_test.drop(columns = ['target','index']).to_numpy()
my_test_y = my_test['target'].to_numpy()
my_pred_x = my_pred.drop(columns = ['target','index']).to_numpy()
my_pred_y = my_pred['target'].to_numpy()
my_pred_y

In [ ]:
print("my_train_x类型："+str(type(my_train_x))+"，my_train_x结构："+str(my_train_x.shape))
print("my_train_y类型："+str(type(my_train_y))+"，my_train_y结构："+str(my_train_y.shape))
print("my_test_x类型："+str(type(my_test_x))+"，my_test_x结构："+str(my_test_x.shape))
print("my_test_y类型："+str(type(my_test_y))+"，my_test_y结构："+str(my_test_y.shape))
print("my_pred_x类型："+str(type(my_pred_x))+"，my_pred_x结构："+str(my_pred_x.shape))
print("my_pred_y类型："+str(type(my_pred_y))+"，my_pred_y结构："+str(my_pred_y.shape))

In [ ]:
gbm = lgb.LGBMRegressor(objective='regression', num_leaves=100, learning_rate=0.05, n_estimators=30)
gbm.fit(my_train_x, my_train_y, eval_set=[(my_test_x, my_test_y)], eval_metric='l1', early_stopping_rounds=5)

In [ ]:
my_result_pred_y = gbm.predict(my_pred_x, num_iteration=gbm.best_iteration_)

In [ ]:
my_pred_y

In [ ]:
my_result_pred_y

In [ ]:
print('The rmse of prediction is:', mean_squared_error(my_pred_y, my_result_pred_y) ** 0.5)
#0.4559142009061406
#0.4559142009061406

# 优化lightGBM模型

In [ ]:
test_year = 2017
pred_year = 2021
#挑选好的特征，再训练
all_vec = ["index","Season","Team1ID","Team2ID","target","T1FGM_mean","T1FGA_mean","T1FGM3_mean","T1FTM_mean","T1FTA_mean","T1OR_mean","T1DR_mean","T1Ast_mean","T1TO_mean","T1Stl_mean","T1Blk_mean","T1PF_mean","T2FGM_mean","T2FGA_mean","T2FGM3_mean","T2FTM_mean","T2FTA_mean","T2OR_mean","T2DR_mean","T2Ast_mean","T2TO_mean","T2Stl_mean","T2Blk_mean","T2PF_mean","T1_seed","T2_seed","seeddiff","T1_rank_mean","T2_rank_mean","rankdiff"]
good_vec = ["T1FGM_mean","T1FGA_mean","T1FGM3_mean","T1FTM_mean","T1FTA_mean","T1OR_mean","T1DR_mean","T1Ast_mean","T1TO_mean","T1Stl_mean","T1Blk_mean","T1PF_mean","T2FGM_mean","T2FGA_mean","T2FGM3_mean","T2FTM_mean","T2FTA_mean","T2OR_mean","T2DR_mean","T2Ast_mean","T2TO_mean","T2Stl_mean","T2Blk_mean","T2PF_mean","seeddiff","rankdiff"]
#good_vec = ["seeddiff"]

In [ ]:
# 新train数据集
good_train = train[(train['Season']!=test_year)&(train['Season']!=pred_year)][["target"]+good_vec].copy().reset_index()
# 新test数据集
good_test = train[train['Season']==test_year][["target"]+good_vec].copy().reset_index()
# predict数据集
good_pred = train[train['Season']==pred_year][["target"]+good_vec].copy().reset_index()


In [ ]:
#做手脚，看模型是否有效
#good_train['seeddiff'] = good_train['target']
#good_test['seeddiff'] = good_test['target']
#good_pred['seeddiff'] = good_pred['target']

#增加命中率

def rate1(series):
    series['T1FG_mean']=series['T1FGM_mean']/series['T1FGA_mean']*100
    series['T2FG_mean']=series['T2FGM_mean']/series['T2FGA_mean']*100
    return series

good_train = good_train.apply(rate1,axis=1)
good_test = good_test.apply(rate1,axis=1)
good_pred = good_pred.apply(rate1,axis=1)

good_pred[['T1FG_mean','T1FGM_mean','T1FGA_mean','T2FG_mean','T2FGM_mean','T2FGA_mean','target']].head()

In [ ]:
good_train.head()

In [ ]:
#pandas转numpy
good_train_x = good_train.drop(columns = ['target','index']).to_numpy()
good_train_y = good_train['target'].to_numpy()
good_test_x = good_test.drop(columns = ['target','index']).to_numpy()
good_test_y = good_test['target'].to_numpy()
good_pred_x = good_pred.drop(columns = ['target','index']).to_numpy()
good_pred_y = good_pred['target'].to_numpy()


In [ ]:
good_train_x[:10]

In [ ]:
good_gbm = lgb.LGBMRegressor(objective='regression', num_leaves=50, learning_rate=0.05, n_estimators=30)
good_gbm.fit(good_train_x, good_train_y, eval_set=[(good_test_x, good_test_y)], eval_metric='l1', early_stopping_rounds=5)

In [ ]:
good_result_pred_y = good_gbm.predict(good_pred_x, num_iteration=good_gbm.best_iteration_)

In [ ]:
#只用seeddiff:0.47112670374089427
#全部直接来：0.4081410823699794
#增加命中率：0.38915333089226206
print('The rmse of prediction is:', mean_squared_error(good_pred_y, good_result_pred_y) ** 0.5)

In [ ]:
good_pred_y

In [ ]:
good_result_pred_y